In [13]:
import os
import pathlib

import geopandas as gpd
import rioxarray as rxr
import xarray as xr

In [14]:
# Define and create the project data directories
data_dir = os.path.join(
    pathlib.Path.home(),
    'GitHub',
    'rivercane_data'
)

os.makedirs(data_dir, exist_ok=True)

In [15]:
# Retrieve location data
%store -r

In [44]:
# Get climate model info from The Climate Model Intercomparison Project
# First make coordinates compatible with shapefile
def convert_longitude(longitude):
    """Convert longitude range from 0-360 to -180-180"""
    return (longitude -360) if longitude > 180 else longitude

ok_miroc5_da_list = []
for site_gdf in [cherokee_county]:
            miroc5_url = (
                'http://thredds.northwestknowledge.net:8080'
                '/thredds/dodsC/MACAV2/MIROC5/'
                'macav2metdata_pr_MIROC5_r1i1p1_rcp45_'
                '2096_2099_CONUS_monthly.nc')
            ok_miroc5_da = xr.open_dataset(miroc5_url).squeeze().precipitation
            bounds = cherokee_county.to_crs(ok_miroc5_da.rio.crs).total_bounds
            bounds
            ok_miroc5_da = ok_miroc5_da.assign_coords(
                lon=("lon", 
                     [convert_longitude(l) for l in ok_miroc5_da.lon.values]))
            ok_miroc5_da = ok_miroc5_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
            ok_miroc5_da.rio.clip_box (*bounds) 
            ok_miroc5_da_list.append(ok_miroc5_da)
ok_miroc5_da_list

[<xarray.DataArray 'precipitation' (time: 48, lat: 585, lon: 1386)> Size: 156MB
 [38918880 values with dtype=float32]
 Coordinates:
   * lat      (lat) float64 5kB 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
     crs      int32 4B 1
   * time     (time) object 384B 2096-01-15 00:00:00 ... 2099-12-15 00:00:00
   * lon      (lon) float64 11kB -124.8 -124.7 -124.7 ... -67.15 -67.11 -67.06
 Attributes:
     long_name:      Monthly Precipitation Amount
     units:          mm
     grid_mapping:   crs
     standard_name:  precipitation
     cell_methods:   time: sum(interval: 24 hours): sum over days
     comments:       Total monthly precipitation at surface: includes both liq...
     _ChunkSizes:    [  8 115 276]]